In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [30]:
%cd '/content/drive/My Drive'

/content/drive/My Drive


In [0]:
import bs4
import requests
import re
import os
import sys

def get_profile_details(profile_tag):
    profile_url = None
    if profile_tag['href']:
        profile_url = 'https://www.amazon.in'+profile_tag['href']
    customer_name_tag = profile_tag.find('span', class_='a-profile-name')
    customer_name = None
    if customer_name_tag:
        customer_name = customer_name_tag.getText()
    return {'profile_url':profile_url, 'customer_name':customer_name}

def get_review_from_tag(review_tag):
    data_dict = dict()
    profile_tag = review_tag.find('a', class_='a-profile')
    if profile_tag:
        data_dict['customer_profile'] = get_profile_details(profile_tag)
    
    review_rating_tag = review_tag.find('i', class_='review-rating')
    if review_rating_tag:
        data_dict['review_rating'] = float(review_rating_tag.getText()[:2])
    
    review_title_tag = review_tag.find('a',  class_='review-title')
    if review_title_tag:
        data_dict['review_title'] = review_title_tag.getText().replace('\n', '  ')
    
    review_date_tag = review_tag.find('span', class_='review-date')
    if review_date_tag:
        data_dict['review_date'] = review_date_tag.getText()
    
    review_text_tag = review_tag.find('span', class_='review-text')
    if review_text_tag:
        data_dict['review_text'] = review_text_tag.getText().replace('\n', '  ')
    
    return data_dict


def get_reviews(url):
    soup = bs4.BeautifulSoup(requests.get(url).text, 'lxml')
    review_tags = soup.find_all('div', id=re.compile('^customer_review'))
    review_data =list()
    for review_tag in review_tags:
        review_data.append(get_review_from_tag(review_tag))
    return review_data




#READ TECHNICAL DETAILS

def read_tbody(tbody_tag):
    data = dict()
    for tr in tbody_tag.find_all('tr'):
        if tr.find('td', class_='label'):
            data[tr.find('td', class_='label').getText().replace('\n', '')] = tr.find('td', class_='value').getText().replace('\n', '')
    return data

def get_techDetails(soup):
    prodDetails_tag = soup.find('div', id='prodDetails')
    techDetails_tag = prodDetails_tag.find('div', class_='techD')
    tech_details = read_tbody(techDetails_tag.table.tbody)
    return tech_details



# PRODUCT DESCRIPTION

def get_prodDescription(soup):
    data = dict()
    disclaim_tag = soup.find('div', class_='disclaim')
    if disclaim_tag:
        for row in disclaim_tag.text.replace('\n', '').strip().split('|'):
            data[row.split(':')[0].strip()] = row.split(':')[0].strip()
    return data

#PRODUCT TITLE
def get_prodTitle(soup):
    prodTitle=soup.find('div', id='titleSection').text.replace('\n', '').strip()
    return prodTitle

def get_customerReviews(soup):
    avg_customer_reviews_tag = soup.find('div', id='averageCustomerReviews')
    number_of_reviews = avg_customer_reviews_tag.find('span', id='acrCustomerReviewText').text.replace('\n', '').strip()
    avg_customer_review = avg_customer_reviews_tag.find('span', id='acrPopover').text.replace('\n', '').strip()
    return {'number_of_reviews':number_of_reviews, 'avg_customer_review':avg_customer_review}

# PRODUCT DEAL PRICE
def get_price(soup):
    p_=soup.find('tr',id=re.compile('priceblock_dealprice')).text.replace('\n', '').strip().split(':')
    return {p_[0]: re.search('([0-9,]+)\\.', p_[1]).group(1).replace(',', '')}

# GET PRODUCT IMAGE URL
def get_imageUrl(soup):
    return soup.find('img', class_='a-dynamic-image a-stretch-vertical')['src']

# SCRAP PRODUCT DETAILS
def scrap_prodDetails(soup):
    data = dict()
    data['Title']=get_prodTitle(soup)
    data['price']=get_price(soup)
    data['image']={'url':get_imageUrl(soup)}
    data['reviews']=get_customerReviews(soup)
    data['product_description']=get_prodDescription(soup)
    data['technical_details']=get_techDetails(soup)
    return data

  
def print_progress(count):
  # Percentage completion.
  # Status-message.
  # Note the \r which means the line should overwrite itself.
  msg = "\r- Progress: {}".format(count)
  # Print it.
  sys.stdout.write(msg)
  sys.stdout.flush()

In [0]:
urls = {'most_recent':
'https://www.amazon.in/OnePlus-Midnight-Black-128GB-Storage/product-reviews/B07DJHY82F/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={}'
,
'top_rated':
'https://www.amazon.in/OnePlus-Midnight-Black-128GB-Storage/product-reviews/B07DJHY82F/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=helpful&pageNumber={}'
,
'battery_life':
{    'most_recent':
    'https://www.amazon.in/OnePlus-Midnight-Black-128GB-Storage/product-reviews/B07DJHY82F/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&filterByKeyword=battery+life&sortBy=recent&pageNumber={}#reviews-filter-bar'
,
    'top_rated':
    'https://www.amazon.in/OnePlus-Midnight-Black-128GB-Storage/product-reviews/B07DJHY82F/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&filterByKeyword=battery+life&sortBy=helpful&pageNumber={}#reviews-filter-bar'
},
 
'picture_quality':
 {   'top_rated':
    'https://www.amazon.in/OnePlus-Midnight-Black-128GB-Storage/product-reviews/B07DJHY82F/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&filterByKeyword=picture+quality&sortBy=helpful&pageNumber={}#reviews-filter-bar'
,
    'most_recent':
    'https://www.amazon.in/OnePlus-Midnight-Black-128GB-Storage/product-reviews/B07DJHY82F/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&filterByKeyword=picture+quality&sortBy=recent&pageNumber={}#reviews-filter-bar'
},

'value_for_money':
{
    'most_recent':'https://www.amazon.in/OnePlus-Midnight-Black-128GB-Storage/product-reviews/B07DJHY82F/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&filterByKeyword=value+for+money&sortBy=recent&pageNumber={}#reviews-filter-bar'
},

'sound_quality':
{
    'most_recent':'https://www.amazon.in/OnePlus-Midnight-Black-128GB-Storage/product-reviews/B07DJHY82F/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&filterByKeyword=sound+quality&sortBy=recent&pageNumber={}#reviews-filter-bar'
},

'fingerprint':
{
    'most_recent':'https://www.amazon.in/OnePlus-Midnight-Black-128GB-Storage/product-reviews/B07DJHY82F/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&filterByKeyword=fingerprint&sortBy=recent&pageNumber={}#reviews-filter-bar'
}   
        
}

In [0]:
import json

In [0]:
url_in = "https://www.amazon.in/dp/B07DJHY82F/ref=gbph_img_m-5_d182_b23b14bf?smid=A23AODI1X2CEAE&pf_rd_p=a3a8dc53-aeed-4aa1-88bb-72ce9ddad182&pf_rd_s=merchandised-search-5&pf_rd_t=101&pf_rd_i=1389401031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_r=P3FSQH2KEB3B5QQ1NQD5"
soup = bs4.BeautifulSoup(requests.get(url_in).text,"lxml")
prod_details = scrap_prodDetails(soup)

In [0]:
with open('product_details.json', 'w') as f:
  json.dump(prod_details, f)
  

In [24]:
import time
not_found = list()
found = []
with open('most_recent_old_1200.json', 'w') as f:
    i = 1
    j = 0
    while j < 100 :
      data = get_reviews(urls['most_recent'].format(i))
      not_found.append(i)      
      i+=1
      for d in data:
        found.append(d)
        j+=1
        print_progress(j)
    json.dump(found, f)

- Progress: 100

In [25]:
not_found = list()
found = []
with open('batterylife.json','w') as f:
    i = 1
    j = 0
    while j < 100 :
      data = get_reviews(urls['battery_life']['most_recent'].format(i))
      not_found.append(i)      
      i+=1
      for d in data:
        found.append(d)
        j+=1
        print_progress(j)
    json.dump(found, f)

- Progress: 100

In [26]:
not_found = list()
found = []
with open('picturequality.json','w') as f:
    i = 1
    j = 0
    while j < 100 :
      data = get_reviews(urls['picture_quality']['most_recent'].format(i))
      not_found.append(i)      
      i+=1
      for d in data:
        found.append(d)
        j+=1
        print_progress(j)
    json.dump(found, f)

- Progress: 100

In [27]:
not_found = list()
found = []
with open('soundquality.json','w') as f:
    i = 1
    j = 0
    while j < 100 :
      data = get_reviews(urls['sound_quality']['most_recent'].format(i))
      not_found.append(i)      
      i+=1
      for d in data:
        found.append(d)
        j+=1
        print_progress(j)
    json.dump(found, f)

- Progress: 100

In [28]:
not_found = list()
found = []
with open('fingerprint.json','w') as f:
    i = 1
    j = 0
    while j < 100 :
      data = get_reviews(urls['fingerprint']['most_recent'].format(i))
      not_found.append(i)      
      i+=1
      for d in data:
        found.append(d)
        j+=1
        print_progress(j)
    json.dump(found, f)

- Progress: 100

In [29]:
not_found = list()
found = []
with open('valueofmoney.json','w') as f:
    i = 1
    j = 0
    while j < 100 :
      data = get_reviews(urls['value_for_money']['most_recent'].format(i))
      not_found.append(i)      
      i+=1
      for d in data:
        found.append(d)
        j+=1
        print_progress(j)
    json.dump(found, f)

- Progress: 100